## Домашняя работа к уроку №3

1. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log (как вариант - использовать np.clip или np.where).

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def calc_logloss(y, p):
    p = np.clip(p, 0.00001, 0.99999)
    err = np.mean(- y * np.log(p) - (1.0 - y) * np.log(1.0 - p))
    return err

2. На данных из урока изучите влияние гиперпараметров на ошибку алгоритма. Подберите аргументы функции eval_LR_model для логистической регрессии таким образом, чтобы log loss не превышал значение 0.3 (0.1). Как изменились веса?

Введем данные с урока:

In [3]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],  # стаж репетитора
              [500, 700, 750, 600, 1450,        # средняя стоимость занятия
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 1, 3, 3, 1, 2]],  # квалификация репетитора
              dtype = np.float64).T 

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1]) # поступил или нет ученик на специальность Математика
X

array([[1.00e+00, 1.00e+00, 5.00e+02, 1.00e+00],
       [1.00e+00, 1.00e+00, 7.00e+02, 1.00e+00],
       [1.00e+00, 2.00e+00, 7.50e+02, 2.00e+00],
       [1.00e+00, 1.00e+00, 6.00e+02, 1.00e+00],
       [1.00e+00, 3.00e+00, 1.45e+03, 2.00e+00],
       [1.00e+00, 0.00e+00, 8.00e+02, 1.00e+00],
       [1.00e+00, 5.00e+00, 1.50e+03, 3.00e+00],
       [1.00e+00, 1.00e+01, 2.00e+03, 3.00e+00],
       [1.00e+00, 1.00e+00, 4.50e+02, 1.00e+00],
       [1.00e+00, 2.00e+00, 1.00e+03, 2.00e+00]])

Определим необходимые функции для расчета весов по алгоритму логистической регрессии:

In [4]:
#функция расчета сигмоиды
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

#функция стандартизации признаков
def standardization(x: np.ndarray) -> np.ndarray:
    s = (x - x.mean()) / x.std()
    return s

# логистическая регрессия
def eval_LR_model(X, y, iterations, eta=1e-4):
    np.random.seed(42)
    w = np.random.randn(X.shape[1])
    n = X.shape[0]
    for i in range(1, iterations + 1):
        z = np.dot(X, w) # log(p/(1-p))
        pred = sigmoid(z) # p [0, 1]
        err = calc_logloss(y, pred)
        w -= eta * (1/n * np.dot((pred - y), X))
        if i % (iterations / 10) == 0:
            print(i, w, err)
    return w

Стандартизируем признак средней стоимости занятия в выборке:

In [5]:
X_st = X.copy()
X_st[:, 2] = standardization(X[:, 2])
X_st[:, 2]

array([-0.97958969, -0.56713087, -0.46401617, -0.77336028,  0.97958969,
       -0.36090146,  1.08270439,  2.11385144, -1.08270439,  0.05155735])

Подберем гиперпараметры функции eval_LR_model, чтобы logloss не превышал 0.3:

In [6]:
w1 = eval_LR_model(X_st, y, 4500, 1e-1)

450 [-1.39768862 -0.93222166  1.02277565  2.35221197] 0.4597647801288568
900 [-2.54631605 -0.99077064  0.65178699  3.12518707] 0.4137239929669819
1350 [-3.48834929 -1.00945824  0.28722727  3.72320838] 0.38299030931245326
1800 [-4.27794785 -1.0270723  -0.01920407  4.23624356] 0.3611357882733705
2250 [-4.95544751 -1.04802176 -0.27474645  4.69237732] 0.3448188601910388
2700 [-5.54974842 -1.07139367 -0.49227314  5.10546336] 0.33209545616851194
3150 [-6.08120495 -1.09604373 -0.68185364  5.48462477] 0.32180028050064774
3600 [-6.56422098 -1.12122835 -0.85054162  5.83653515] 0.31320978295693064
4050 [-7.00914227 -1.14652166 -1.00323799  6.16623645] 0.3058577945489886
4500 [-7.4235364  -1.17168963 -1.14340807  6.47759198] 0.2994332900954316


И 0.1:

In [7]:
w2 = eval_LR_model(X_st, y, 16000, 1)

1600 [-14.47030246  -1.71822939  -3.36802224  12.07275068] 0.22289419298592222
3200 [-20.83870528  -2.26834263  -5.27602619  17.23786627] 0.17794662172361947
4800 [-25.57711067  -2.68514118  -6.66668782  21.07928423] 0.153237054655682
6400 [-29.36523152  -3.01981408  -7.7663975   24.14425677] 0.13751741721431626
8000 [-32.53553759  -3.29955247  -8.68108548  26.70335505] 0.12654567671147138
9600 [-35.27563914  -3.54022646  -9.46899463  28.90956968] 0.11837363409529396
11200 [-37.7006774   -3.75178214 -10.16522941  30.85687919] 0.11198969688882159
12800 [-39.88590776  -3.94081856 -10.7924227   32.60674563] 0.10681865312927956
14400 [-41.88299586  -4.11192    -11.36594431  34.20138957] 0.10250981063660962
16000 [-43.7288341   -4.26839306 -11.89666961  35.6709789 ] 0.09883723515238044


Видно, что для достижения необходимых значений Logloss надо повышать количество итераций и увеличивать скорость обучения. Причем важно следить за тем, чтобы при повышении eta расчет сходился. При снижении Logloss веса признаков увеличиваются по модулю.

3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса "1". На вход функции подаются значения признаков Х и веса, которые уже посчитаны функцией eval_LR_model.

In [22]:
def calc_pred_proba(X, w):
    z = np.dot(X, w)
    pred = sigmoid(z)
    return pred

w = eval_LR_model(X_st, y, 1000, 1e-1)
np.set_printoptions(floatmode = 'fixed')
print(calc_pred_proba(X_st, w))

100 [-0.29848458 -0.72686866  1.06314016  1.39690737] 0.5235767485993659
200 [-0.64245884 -0.81575707  1.11680356  1.72702881] 0.4995552177931558
300 [-0.95659618 -0.87689661  1.10232809  2.00636498] 0.4814150662731217
400 [-1.25432451 -0.91736333  1.05455726  2.24457861] 0.4664467561141666
500 [-1.53759570 -0.94440497  0.98735478  2.45373702] 0.4535011239613406
600 [-1.80755433 -0.96270374  0.90904148  2.64183838] 0.44201377632900957
700 [-2.06512906 -0.97530110  0.82492184  2.81420765] 0.4316738793739682
800 [-2.31113497 -0.98420861  0.73842971  2.97447927] 0.42228858977623346
900 [-2.54631605 -0.99077064  0.65178699  3.12518707] 0.4137239929669819
1000 [-2.77136565 -0.99588853  0.56641089  3.26813012] 0.4058781527072773
[0.25846265 0.30568870 0.81910473 0.28147361 0.79118436 0.57257036
 0.93504121 0.15077287 0.24742752 0.85843113]


4. Создайте функцию calc_pred, возвращающую предсказанные классы (0 или 1). На вход функции подаются значения признаков Х и веса, которые уже посчитаны функцией eval_LR_model, а также порог вероятности.

In [23]:
def calc_pred(X, w, p):
    z = np.dot(X, w)
    pred = sigmoid(z)
    pred = np.where(pred > p, 1, 0)
    return pred

print(calc_pred(X_st, w, 0.4))

[0 0 1 0 1 1 1 0 0 1]


5. (*) Напишите функции для расчета accuracy, матрицы ошибок, precision и recall, а также F1-score.

In [49]:
#Функция расчета accuracy по истинным и предсказанным значениям
def accuracy(y, y_pred):
    acc = 100.0 - np.mean(np.abs(y_pred - y)*100.0)
    return acc

y_pred = calc_pred(X_st, w, 0.4)
print(accuracy(y, y_pred))

80.0


In [41]:
#Функция, возвращающая матрицу ошибок
def con_matrix(y, y_pred):
    con_matrix = np.zeros((2,2), dtype = 'int16')
    TP, FP, FN, TN = 0, 0, 0, 0
    for i in range(len(y)):
        if y[i] == 1 and y_pred[i] == 1:
            TP +=1
        if y[i] == 1 and y_pred[i] == 0:
            FN +=1
        if y[i] == 0 and y_pred[i] == 1:
            FP +=1
        if y[i] == 0 and y_pred[i] == 0:
            TN +=1
    con_matrix[0, 0] = TP
    con_matrix[1, 0] = FN
    con_matrix[0, 1] = FP
    con_matrix[1, 1] = TN
    return con_matrix

print(con_matrix(y, y_pred))

[[4 1]
 [1 4]]


In [42]:
#Функция, возвращающая метрику precision
def precision(y, y_pred):
    A = con_matrix(y, y_pred)
    precision = A[0,0] / (A[0,0] + A[0,1])
    return precision

print(precision(y, y_pred))

0.8


In [43]:
#Функция, возвращающая метрику recall
def recall(y, y_pred):
    A = con_matrix(y, y_pred)
    recall = A[0,0] / (A[0,0] + A[1,0])
    return recall

print(recall(y, y_pred))

0.8


In [50]:
#Функция, возвращающая метрику F_score
def F_score(y, y_pred):
    F_score = 2 * precision(y, y_pred) * recall(y, y_pred)/(precision(y, y_pred) + recall(y, y_pred))
    return F_score

print(F_score(y, y_pred))

0.800000011920929
